## Import Necessary Packages

In [1]:
import glob
import numpy as np
import xarray as xr
import proplot as pplt
pplt.rc.update({'font.size':12})
import warnings
warnings.filterwarnings('ignore')

## User-Defined Fields

In [2]:
path  = # '/directory/where/large/domain/analysis/files/are/located/'
cases = ['nz_32','nz_64','nz_128']

## Import Binned Data

Load in data binned by column relative humidity. Files are added to ```dslist``` manually due to weird timeframe dimension and that fact that there are multiple files for nz_128.

In [3]:
dslist = []
for case in cases:
    dslist.append(xr.open_dataset(glob.glob(path+'crh_binned_vars_nz_'+case+'_figure_timeframes.nc')[0],
                                  decode_cf=False))

Read in ```timeframes``` and ```bincenter```, and reformat timeframe strings so it has leading zeros.

In [4]:
timeframes = dslist[0].timeframe.values
bincenter  = dslist[0].percentile_center.values
for i,timeframe in enumerate(timeframes):
    # Find index of the dash
    dash  = timeframe.index('-')
    start = int(timeframe[:dash])
    stop  = int(timeframe[dash+1:])
    timeframes[i] = '{:03d}'.format(start)+'-{:03d}'.format(stop)    

## Plot Fig. 7

In [5]:
top='RdBu_r'; bottom='Cubehelix'; linewidth=0.8; qnlevels=[0.003,0.006,0.009,0.012]
dpsi=0.0010; psilevels=np.arange(-0.0110,0.0110+dpsi,dpsi)
fig,axs = pplt.subplots(ncols=3,nrows=2,refwidth=2,refheight=3)
for i,timeframe in enumerate(timeframes):
    axs.format(xlabel='Column relative humidity percentile',xlim=(0,100),xticks=25,ylabel='z (km)',ylim=(0,13),
               abc='a',abcloc='l',suptitle='Days '+timeframe)
    for j,case in enumerate(cases):
        # Plot radiative heating
        a = axs[j].contourf(bincenter,dslist[j].z/1000,np.transpose(dslist[j].qr.isel(timeframe=i).values),
                               cmap=top,levels=20,vmin=-1.5,vmax=1.5,extend='both')
        axs[j].contour(bincenter,dslist[j].z/1000,np.transpose(dslist[j].psi.isel(timeframe=i).values),
                       color='k',linewidth=linewidth,levels=psilevels)
        axs[j].contour(bincenter,dslist[j].z/1000,np.transpose(dslist[j].qn.isel(timeframe=i).values),
                       color='w',linewidth=linewidth,levels=qnlevels)
        axs[j].format(titleloc='l',title=case+', $Q_{R}$') 
        # Plot FMSE
        b = axs[j+3].contourf(bincenter,dslist[j].z/1000,np.transpose(dslist[j].fmse.isel(timeframe=i).values)/1000,
                                 cmap=bottom,levels=21,vmin=315,vmax=335,extend='both')
        axs[j+3].contour(bincenter,dslist[j].z/1000,np.transpose(dslist[j].psi.isel(timeframe=i).values),
                         color='k',linewidth=linewidth,levels=psilevels)
        axs[j+3].contour(bincenter,dslist[j].z/1000,np.transpose(dslist[j].qn.isel(timeframe=i).values),
                         color='w',linewidth=linewidth,levels=qnlevels)
        axs[j+3].format(titleloc='l',title=case+ r', $\langle h_{\mathrm{f}} \rangle$')
    axs[2].colorbar(a,loc='r',label='Radiative heating rate (K day$^{-1}$)')
    axs[5].colorbar(b,loc='r',label='Frozen moist static energy (kJ m$^{-2}$)')